# マルチモーダル　RAG チャットボットの作成

## 以下の順でlangchain実装練習
1. ユーザインタフェースの実装
2. 質問応答システムの拡張
3. 会話履歴の実装
4. コンテキストの拡張
5. RAGの実装
6. マルチモーダル対応

## 1. ユーザインタフェースの実装

In [22]:
import streamlit as st
st.title("マルチモーダルRAGチャットボット")

# アップローダを追加
uploaded_file = st.file_uploader("画像を選択してください", type=["jpg", "jpeg", "png"])
# アップロードされた画像を表示
if uploaded_file is not None:
    st.image(uploaded_file, caption="画像", widch=300)

# ユーザ入力を受け取る
user_input = st.text_input("メッセージを入力してください:") 

#　ボタンを追加し、クリックしたらアクションを起こす
if st.button("送信"):
#　入力されたテキストを表示
    st.write(f"human: {user_input}")


2025-11-11 02:18:38.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.616 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.619 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-11 02:18:38.619 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

## 2. 質問応答システムへの拡張

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key,   # ←ここでキーを渡す
    )

response = llm.invoke(user_input)
    # 会話を表示
st.write(f"ai: {response.content}")


In [17]:
# 接続テスト
print(llm.invoke("1+1は？").content)

1+1は**2**です。


## 3. 会話履歴の実装

In [ ]:
from langchain_core.messages import HumanMessage
# セッション状態を初期化
if "history" not in st.session_state:
    st.session_state.history = []
    st.session_state.llm = ChatGoogleGenerativeAI()
    

- Streamlitは“実行がリロード型”、継続したい情報はst.session_stateに置く必要がある。

In [ ]:
# ボタンを追加してクリックされたらアクションを起こす
if st.button("送信"):
    st.session_state.history.append(HumanMessage(user_input))
    response = st.session_state.llm.invoke(st.session_state.history)
    st.session_state.history.append(response)    

    # 会話を表示
    for message in reversed(st.session_state.history):
        st.write(f"{message.type}: {message.content}")

- 会話がAIと人間で交互にhistoryに保存されるようになる
- それを逆順で表示（新しいのが最新一番上にくる）

## 4. コンテキストの拡張

In [2]:
from langchain_core.prompts import ChatPromptTemplate

ModuleNotFoundError: No module named 'langchain_core'

In [ ]:
# チェーンを作成
def create_chain():
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "回答には以下の情報も参考にしてください。参考情報：\n{info}",
            ),
            ("placeholder", "{history}"),
            ("human", "{input}"),
        ]
    )
    return prompt | ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key)
        

- プロンプトテンプレートで入力データを整形してから、モデルに渡して応答を得る

- 左の prompt：入力（info, history, input など）を受け取り→ LLMが読めるメッセージ形式に変換
- 右の ChatGoogleGenerativeAI：それを受け取って→ 実際にAPIで生成処理（推論）を行う

- placeholderは過去の会話履歴そのまま挿入するよってイメージ？


In [ ]:
# クリック時のアクションを変更してみる
response = st.session_state.chain.invoke(
    {
        "input":user_input,
        "history":st.session_state.history,
        "info":"ユーザの年齢は10歳です",
    }
)

## 5. RAGの実装

## インデックスを作成する

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
def load_document(filename):
    # CSVをロード
    loader = CSVLoader(file_path=filename, autodetect_encoding=True)
    pages = loader.load()

    # テキストを分割
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
    splits = text_splitter.split_documents(pages)

    # 埋め込みモデルを定義（Gemini埋め込み）
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    # Chromaベクトルストアを作成・保存
    db = Chroma.from_documents(
        documents=splits,
        embedding=embeddings,
        persist_directory="data/hino_trash"  # ベクトルデータを保存する場所
    )
    print("インデックス作成完了")

# ← ここから下がスクリプト直実行時の入り口
if __name__ == "__main__":
    # ここでCSVの場所を指定（相対パス or 絶対パス）
    csv_path = "../../data/hino_trash/hino_trash.csv"   # ← 自分のファイル名に合わせて変更
    load_document(csv_path)

### 一度だけ実行してインデックス作成すれば良い
- docker exec -it project_api-streamlit bash
- cd /work/src/app
- python make_index.py

### これでようやくできた

### インデックスの利用

In [6]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from operator import itemgetter

ModuleNotFoundError: No module named 'langchain_chroma'

In [ ]:
# ドキュメントを整形
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [7]:
def create_chain():
    vectorstore = Chroma(
        embedding_function=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
        persist_directory="data/hino_trash",
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    

In [ ]:
    return(
        {
            "input": itemgetter("input"),
            "info": itemgetter("input") | retriever | format_docs,
            "history": itemgetter("history"),
        }
        | prompt 
        | ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=api_key, temperature=0)
    )

## 6. マルチモーダル対応

### 画像とテキストで検索